In [41]:
%config IPCompleter.greedy=True
%load_ext memory_profiler
%matplotlib inline
%run DSF_FINAL_PROJECT_Helper.ipynb
%run DSF_FINAL_PROJECT_Helper_General.ipynb
%run DSF_FINAL_PROJECT_Plot.ipynb

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


INFO - 10:33:27: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 10:33:27: built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)


### Description

In this notebook we are going to create and train Word2Vec model (using a dataset from https://ieee-dataport.org/open-access/coronavirus-covid-19-tweets-dataset).

With this notebook we will implement the "transfer learning" method (https://en.wikipedia.org/wiki/Transfer_learning). So, the Word2Vec model will be the starting point of our unsupervised learning model (since we extract tweets directly without labelling)

<h1 style="background-color:powderblue;">Twitter Training Dataset Ingestion</h1>

In [2]:
# We getting the rated (labelled) dataset from 
# https://ieee-dataport.org/open-access/coronavirus-covid-19-tweets-dataset

In [3]:
#file = pd.read_csv("corona_tweets_296.csv")
#df_user = pd.read_csv('corona_tweets_296.csv', index_col='user_id', delimiter='|')
df_corona_tweets_train_ori = pd.read_csv('corona_tweets_296.csv', delimiter=',', header=None, names=['tweet_id ', 'rate'])

In [4]:
df_corona_tweets_train_ori.head(3)

,tweet_id,rate
0,1347399059334516736,0.136905
1,1347399059708010496,0.000000
2,1347399059921891328,0.000000


In [5]:
df_corona_tweets_train_ori.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2975849 entries, 0 to 2975848
Data columns (total 2 columns):
 #   Column     Dtype  
---  ------     -----  
 0   tweet_id   int64  
 1   rate       float64
dtypes: float64(1), int64(1)
memory usage: 45.4 MB


In [6]:
df_corona_tweets_train_filtered = df_corona_tweets_train_ori[df_corona_tweets_train_ori.rate!=0]
df_corona_tweets_train_filtered.reset_index(drop=True, inplace=True)

In [7]:
df_corona_tweets_train_filtered = df_corona_tweets_train_filtered[:1000]

In [8]:
df_corona_tweets_train_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   tweet_id   1000 non-null   int64  
 1   rate       1000 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 15.8 KB


In [9]:
df_corona_tweets_train_filtered.head(3)

,tweet_id,rate
0,1347399059334516736,0.136905
1,1347399060219695104,0.098788
2,1347399060236369922,0.400000


In [10]:
cleaningHelper = CleaningHelper(version = "1.0")
print(cleaningHelper)

Cleaning helper version 1.0


In [11]:
cleaningHelper.get_nulls_data(df_corona_tweets_train_filtered)

,Total,PercNotNull
tweet_id,0,100.0
rate,0,100.0


In [12]:
df_corona_tweets_train_filtered.rate.value_counts()/len(df_corona_tweets_train_filtered)

-0.450000    0.068
 0.136364    0.054
 0.250000    0.045
 0.500000    0.032
-0.100000    0.028
             ...  
 0.162476    0.001
 0.102778    0.001
 0.016667    0.001
-0.273148    0.001
-0.080000    0.001
Name: rate, Length: 294, dtype: float64

<h3 style="background-color:powderblue;">Data cleaning</h3>

In [13]:
#I decided to use the largest one
spacy_lg = spacy.load('en_core_web_lg')

In [14]:
# sets of punctuation in variable result 
punctuation_str = string.punctuation  
punctuation_str #I want to know if @ is actually include

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [15]:
#Stop words Set
#stop_words = spacy.lang.en.stop_words.STOP_WORDS
stop_words = cleaningHelper.get_custom_stop_words(spacy_ = spacy_lg)

In [16]:
#English parser object
parser = spacy.lang.en.English()

In [17]:
cleaningTweets = CleaningTweets(version = "1.0", spacy_ = spacy_lg, parser_ = parser, punctuation_str_ = punctuation_str, stop_words_ = stop_words)
print(cleaningTweets)

Cleaning Tweets version 1.0


In [18]:
twitterAuthentication = TwitterAuthentication(version = "1.0")
api = twitterAuthentication.connect()

In [50]:
#Connecting to sqlite3 database
dbConn = Sqlite3Db('social_network.db')

In [20]:
#Creating tweets table
dbConn.query('''CREATE TABLE IF NOT EXISTS TweetsTraining(insert_timestamp timestamp, tweet_id TEXT, rate TEXT, tweet_text TEXT)''')

In [21]:
CLEAN_TRAINING = False
#Cleaning TweetsTraining table
if CLEAN_TRAINING:
    del_stm_str = """DELETE FROM TweetsTraining;"""
    dbConn.query(sqlStm=del_stm_str)

In [22]:
twitterProcess = TwitterProcess(api = api, dbConn = dbConn)
limit, remaining, next_reset_time = twitterProcess.get_api_limits()
print(limit, remaining, next_reset_time)

180 180 2021-01-10 10:38:36


In [23]:
#a = twitterProcess.get_tweet_text_by_id(tweet_id_ = "1347399060739796992")
#a = twitterProcess.get_tweet_text_by_id(tweet_id_ = "1347399060219695104")

In [24]:
#a

In [25]:
#status = api.get_status(1347399060219695104)
#status = status.text
#status

In [26]:
#start = time.time()
#start

In [27]:
sqlite3DbHelper = Sqlite3DbHelper(dbConn = dbConn)
#query_stm_res = sqlite3DbHelper.get_count_stm(sql_stm_res)
#query_stm_res

In [28]:
#for index, row in df_corona_tweets_train_filtered.iterrows():
#    #print(index)
#    print(row[1])
#    rate_ = f"{row[1]:.0f}"

In [29]:
if CLEAN_TRAINING:
    start = time.time()
    tweet_text_list = []
    for index, row in df_corona_tweets_train_filtered.iterrows():
        #print(index)
        #print(row[0])

        tweet_id_ = f"{row[0]:.0f}"
        rate_ = f"{row[1]:.16f}"
        #print(f"{tweet_id_}")
        #if index == 9:
        #    break

        #print(tweet_id_)
        #print(twitterProcess.get_tweet_text_by_id(tweet_id_ = tweet_id_))
        twitter_status_ = twitterProcess.get_tweet_text_by_id(tweet_id_ = tweet_id_)
        ins_stm_str = """INSERT INTO TweetsTraining (insert_timestamp, tweet_id, rate, tweet_text) VALUES (?,?,?,?);"""
        #if len(twitter_status_.strip()) > 0:

        insert_timestamp = sqlite3DbHelper.get_timestamp_now()

        data_tuple = (insert_timestamp, tweet_id_, rate_, twitter_status_)
        dbConn.query(sqlStm=ins_stm_str, sqlStmPrm=data_tuple)
        sql_stm_res = dbConn.cursor.fetchall()
        tweet_text_list.append(twitter_status_)

    print('Before End time: {} mins'.format(round((time.time() - start) / 60, 2)))
    df_corona_tweets_train_filtered["tweet_text"] = tweet_text_list
    print('End time: {} mins'.format(round((time.time() - start) / 60, 2)))

In [30]:
#try:
#    tweet_text_ = twitterProcess.get_tweet_text_by_id(tweet_id_ = "1347399060739796992")
#    print(tweet_text_)
#except tweepy.TweepError as e:
#    print(e.message[0]['code'])
#    print(e.args[0][0]['code'])
#    #print("Error")

In [31]:
sel_tt_stm_str = '''SELECT insert_timestamp, tweet_id, rate, tweet_text FROM TweetsTraining WHERE length(trim(tweet_text)) > 0;'''

df_tweets_training_sql_ori = pd.read_sql_query(sel_tt_stm_str, dbConn.conn)

In [32]:
df_tweets_training_sql_ori.head(3)

,insert_timestamp,tweet_id,rate,tweet_text
0,2021-01-10 08:23:14.641840,1347399060219695104,0.0987878787878788,"RT @nytimes: The U.S. reported 4,000 coronavir..."
1,2021-01-10 08:23:15.512763,1347399060341272576,0.1821428571428571,RT @FOX4: The U.S. registered more COVID-19 de...
2,2021-01-10 08:23:15.865122,1347399060588818432,0.1666666666666666,"RT @NewYearsDani: if you're angry, stop spendi..."


In [33]:
df_tweets_training_sql_cleaned = df_tweets_training_sql_ori.copy()
df_tweets_training_sql_cleaned["tweet_vector"] = df_tweets_training_sql_cleaned["tweet_text"] 

In [34]:
df_tweets_training_sql_cleaned["tweet_vector"] = df_tweets_training_sql_cleaned["tweet_text"].apply(lambda x: cleaningTweets.do_spacy_tokenizer(token_ = x) )

In [37]:
df_tweets_training_sql_cleaned.head(3)

,insert_timestamp,tweet_id,rate,tweet_text,tweet_vector
0,2021-01-10 08:23:14.641840,1347399060219695104,0.0987878787878788,"RT @nytimes: The U.S. reported 4,000 coronavir...","[u.s., reported, 4,000, coronavirus, deaths, t..."
1,2021-01-10 08:23:15.512763,1347399060341272576,0.1821428571428571,RT @FOX4: The U.S. registered more COVID-19 de...,"[u.s., registered, covid-19, deaths, single, d..."
2,2021-01-10 08:23:15.865122,1347399060588818432,0.1666666666666666,"RT @NewYearsDani: if you're angry, stop spendi...","[angry, stop, spending, afraid, stop, spending..."


In [38]:
df_tweets_training_sql_model = df_tweets_training_sql_cleaned.copy()

In [39]:
df_tweets_training_sql_model = df_tweets_training_sql_model[df_tweets_training_sql_model.tweet_vector.str.len()>1]

In [40]:
df_tweets_training_sql_model.head(3)

,insert_timestamp,tweet_id,rate,tweet_text,tweet_vector
0,2021-01-10 08:23:14.641840,1347399060219695104,0.0987878787878788,"RT @nytimes: The U.S. reported 4,000 coronavir...","[u.s., reported, 4,000, coronavirus, deaths, t..."
1,2021-01-10 08:23:15.512763,1347399060341272576,0.1821428571428571,RT @FOX4: The U.S. registered more COVID-19 de...,"[u.s., registered, covid-19, deaths, single, d..."
2,2021-01-10 08:23:15.865122,1347399060588818432,0.1666666666666666,"RT @NewYearsDani: if you're angry, stop spendi...","[angry, stop, spending, afraid, stop, spending..."


In [42]:
sent = [row for row in df_tweets_training_sql_model.tweet_vector]
phrases = Phrases(sent, min_count=1, progress_per=50000)
bigram = Phraser(phrases)
sentences = bigram[sent]
sentences[2]

INFO - 10:33:55: collecting all words and their counts
INFO - 10:33:55: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 10:33:55: collected 4247 word types from a corpus of 3688 words (unigram + bigrams) and 436 sentences
INFO - 10:33:55: using 4247 counts as vocab in Phrases<0 vocab, min_count=1, threshold=10.0, max_vocab_size=40000000>
INFO - 10:33:55: source_vocab length 4247
INFO - 10:33:55: Phraser built with 253 phrasegrams


['u.s.',
 'registered',
 'covid-19_deaths',
 'single_day',
 '--',
 'nearly',
 '3,900',
 '--',
 'day',
 'mob',
 'attack']

In [46]:
w2v_model = Word2Vec(min_count=3,
                     window=4,
                     size=300,
                     sample=1e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=multiprocessing.cpu_count()-1)

start = time.time()

w2v_model.build_vocab(sentences, progress_per=50000)

print('Time to build vocab: {} mins'.format(round((time.time() - start) / 60, 2)))

INFO - 10:44:41: collecting all words and their counts
INFO - 10:44:41: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 10:44:41: collected 1882 word types from a corpus of 3067 raw words and 436 sentences
INFO - 10:44:41: Loading a fresh vocabulary
INFO - 10:44:41: effective_min_count=3 retains 187 unique words (9% of original 1882, drops 1695)
INFO - 10:44:41: effective_min_count=3 leaves 1082 word corpus (35% of original 3067, drops 1985)
INFO - 10:44:41: deleting the raw counts dictionary of 1882 items
INFO - 10:44:41: sample=1e-05 downsamples 187 most-common words
INFO - 10:44:41: downsampling leaves estimated 46 word corpus (4.3% of prior 1082)
INFO - 10:44:41: estimated required memory for 187 words and 300 dimensions: 542300 bytes
INFO - 10:44:41: resetting layer weights


Time to build vocab: 0.0 mins


In [47]:
start = time.time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time.time() - start) / 60, 2)))

w2v_model.init_sims(replace=True)

INFO - 10:44:55: training model with 3 workers on 187 vocabulary and 300 features, using sg=0 hs=0 sample=1e-05 negative=20 window=4
INFO - 10:44:56: worker thread finished; awaiting finish of 2 more threads
INFO - 10:44:56: worker thread finished; awaiting finish of 1 more threads
INFO - 10:44:56: worker thread finished; awaiting finish of 0 more threads
INFO - 10:44:56: EPOCH - 1 : training on 3067 raw words (42 effective words) took 0.0s, 1462 effective words/s
INFO - 10:44:56: worker thread finished; awaiting finish of 2 more threads
INFO - 10:44:56: worker thread finished; awaiting finish of 1 more threads
INFO - 10:44:56: worker thread finished; awaiting finish of 0 more threads
INFO - 10:44:56: EPOCH - 2 : training on 3067 raw words (46 effective words) took 0.0s, 1846 effective words/s
INFO - 10:44:56: worker thread finished; awaiting finish of 2 more threads
INFO - 10:44:56: worker thread finished; awaiting finish of 1 more threads
INFO - 10:44:56: worker thread finished; awai

INFO - 10:45:00: worker thread finished; awaiting finish of 1 more threads
INFO - 10:45:00: worker thread finished; awaiting finish of 0 more threads
INFO - 10:45:00: EPOCH - 25 : training on 3067 raw words (44 effective words) took 0.1s, 368 effective words/s
INFO - 10:45:00: worker thread finished; awaiting finish of 2 more threads
INFO - 10:45:00: worker thread finished; awaiting finish of 1 more threads
INFO - 10:45:00: worker thread finished; awaiting finish of 0 more threads
INFO - 10:45:00: EPOCH - 26 : training on 3067 raw words (46 effective words) took 0.1s, 624 effective words/s
INFO - 10:45:00: worker thread finished; awaiting finish of 2 more threads
INFO - 10:45:00: worker thread finished; awaiting finish of 1 more threads
INFO - 10:45:00: worker thread finished; awaiting finish of 0 more threads
INFO - 10:45:00: EPOCH - 27 : training on 3067 raw words (52 effective words) took 0.1s, 1021 effective words/s
INFO - 10:45:00: worker thread finished; awaiting finish of 2 more

Time to train the model: 0.08 mins


In [48]:
w2v_model.save("word2vec.model")

INFO - 10:45:08: saving Word2Vec object under word2vec.model, separately None
INFO - 10:45:08: not storing attribute vectors_norm
INFO - 10:45:08: not storing attribute cum_table
INFO - 10:45:08: saved word2vec.model


In [51]:
#Close database
dbConn.close()